# 개요

# Prepare

# 1. 초기 세팅

## Library & data load

In [289]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [290]:
# 처리용 df와 오리지널 버전 남겨놓음
df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202103.csv",encoding='euc_kr')
org_df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202103.csv",encoding='euc_kr')

# Pre-Processing

## Basic info

In [291]:
df.head()

,품목명,단위,등급,가격,산지,친환경구분,입력일
0,[가리비국산]가리비국산,5kg,보통(3등),23000,경남 남해,일반,20210301
1,[가리비국산]가리비국산,5kg,보통(3등),25000,"경남 마산(고성,진해)",일반,20210301
2,[가리비국산]가리비국산,5kg,보통(3등),24000,"경남 마산(고성,진해)",일반,20210301
3,[가오리수입]가오리수입,10kg,보통(3등),51230,수입 아르헨티나,일반,20210301
4,[가자미국산]가자미국산,15kg,보통(3등),162000,국내산,일반,20210301


In [292]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490992 entries, 0 to 490991
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   품목명     490992 non-null  object
 1   단위      490992 non-null  object
 2   등급      490992 non-null  object
 3   가격      490992 non-null  int64 
 4   산지      489074 non-null  object
 5   친환경구분   490992 non-null  object
 6   입력일     490992 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 26.2+ MB


In [293]:
# 산지 제외하고는 결측치 없는 것으로 확인
df.isnull().sum()

품목명         0
단위          0
등급          0
가격          0
산지       1918
친환경구분       0
입력일         0
dtype: int64

- 결측치 처리

In [294]:
###
# 양이 많지 않으므로 결측치 제거
df = df.dropna(axis=0)

- 영문명으로 전환

In [295]:
df = df.rename(columns = {'품목명':'prd',
                          '단위':'scale','등급':'class',
                          '가격':'price',
                          '산지':'origin',
                          '친환경구분':'eco',
                          '입력일':'reg_date'})

## column 별 정보

### class

In [296]:
# 이름 모두 영어로 변경
# 9등과 없음 모두 9로 변경
df['class'].value_counts().index

Index(['특(1등)', '상(2등)', '보통(3등)', '9등(등외)', '없음', '5등', '4등', '6등', '7등',
       '8등'],
      dtype='object')

In [297]:
df111 = pd.DataFrame([['특(1등)', 1], ['상(2등)',2], ['보통(3등)',3], ['9등(등외)',9], ['없음',9], ['5등',5], ['4등',4], ['6등',6], ['7등',7],['8등',8]],columns=['class','new_class'])

In [298]:
df2 = df.merge(df111)
df2 = df2.drop(columns='class')

In [299]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가리비국산]가리비국산,5kg,23000,경남 남해,일반,20210301,3
1,[가리비국산]가리비국산,5kg,25000,"경남 마산(고성,진해)",일반,20210301,3
2,[가리비국산]가리비국산,5kg,24000,"경남 마산(고성,진해)",일반,20210301,3
3,[가오리수입]가오리수입,10kg,51230,수입 아르헨티나,일반,20210301,3
4,[가자미국산]가자미국산,15kg,162000,국내산,일반,20210301,3
...,...,...,...,...,...,...,...
489069,[토마토]토마토(기타),2.5kg,5500,부산 강서구,일반,20210330,7
489070,[감귤]감귤(금귤),.5kg,1100,제주 서귀포,일반,20210331,7
489071,[감귤]감귤(금귤),10kg,14000,제주 서귀포,일반,20210331,7
489072,[당근]당근(당근),20kg,9000,제주 제주시,일반,20210331,7


### eco

In [300]:
# 영문명으로 전환
# 전환기 -> 유기농 재배로 전환중
df2.loc[df2["eco"]=="일반","eco"] = "normal"
df2.loc[df2["eco"]=="우수농산물","eco"] = "good_prd"
df2.loc[df2["eco"]=="무농약","eco"] = "ecofriendly"
df2.loc[df2["eco"]=="유기농","eco"] = "organic"
df2.loc[df2["eco"]=="품질인증","eco"] = "certified"
df2.loc[df2["eco"]=="전환기","eco"] = "ing"

In [301]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가리비국산]가리비국산,5kg,23000,경남 남해,normal,20210301,3
1,[가리비국산]가리비국산,5kg,25000,"경남 마산(고성,진해)",normal,20210301,3
2,[가리비국산]가리비국산,5kg,24000,"경남 마산(고성,진해)",normal,20210301,3
3,[가오리수입]가오리수입,10kg,51230,수입 아르헨티나,normal,20210301,3
4,[가자미국산]가자미국산,15kg,162000,국내산,normal,20210301,3
...,...,...,...,...,...,...,...
489069,[토마토]토마토(기타),2.5kg,5500,부산 강서구,normal,20210330,7
489070,[감귤]감귤(금귤),.5kg,1100,제주 서귀포,normal,20210331,7
489071,[감귤]감귤(금귤),10kg,14000,제주 서귀포,normal,20210331,7
489072,[당근]당근(당근),20kg,9000,제주 제주시,normal,20210331,7


### reg_date

In [302]:
df2['reg_date'] = df2['reg_date'].astype('str')

In [303]:
df2['reg_date'] = pd.to_datetime(df2['reg_date'].astype('str'),format='%Y%m%d')

In [304]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가리비국산]가리비국산,5kg,23000,경남 남해,normal,2021-03-01,3
1,[가리비국산]가리비국산,5kg,25000,"경남 마산(고성,진해)",normal,2021-03-01,3
2,[가리비국산]가리비국산,5kg,24000,"경남 마산(고성,진해)",normal,2021-03-01,3
3,[가오리수입]가오리수입,10kg,51230,수입 아르헨티나,normal,2021-03-01,3
4,[가자미국산]가자미국산,15kg,162000,국내산,normal,2021-03-01,3
...,...,...,...,...,...,...,...
489069,[토마토]토마토(기타),2.5kg,5500,부산 강서구,normal,2021-03-30,7
489070,[감귤]감귤(금귤),.5kg,1100,제주 서귀포,normal,2021-03-31,7
489071,[감귤]감귤(금귤),10kg,14000,제주 서귀포,normal,2021-03-31,7
489072,[당근]당근(당근),20kg,9000,제주 제주시,normal,2021-03-31,7


### price

In [305]:
# 계산을 위해 scale 값 미리 변경
# 현재 값들은 string이므로 float 형태로 변환할 필요가 있음
# 몇몇 이상치 처리도 진행
def scale_checker(scale):
  if scale.startswith('.')==True:
    return format(float('0'+scale[:-2]),'.2f')
  else:
    return format(float((scale[:-2])),'.2f')
df2['scale'] = df2['scale'].apply(scale_checker)
df2['scale'] = df2['scale'].astype('float')
# df2['scale'] = format(round(df2['scale'].apply(pd.to_numeric),2),'.2f')

In [306]:
df2['scale'].unique()

array([   5.  ,   10.  ,   15.  ,    8.  ,    5.01,   20.  ,    2.4 ,
          6.  ,    2.  ,    3.  ,    0.05,    0.2 ,    0.9 ,    1.  ,
          0.8 ,    0.7 ,    4.  ,    1.7 ,    1.8 ,    0.5 ,   23.  ,
          7.5 ,    4.5 ,    9.  ,    1.6 ,    1.2 ,    1.5 ,    1.4 ,
         19.  ,   12.  ,    1.9 ,   18.  ,    3.5 ,    2.5 ,    9.58,
          7.  ,    5.4 ,    2.2 ,    2.1 ,    5.1 ,    3.4 ,    3.3 ,
          3.8 ,    4.2 ,    9.6 ,    2.7 ,   11.  ,    2.3 ,    9.51,
         21.5 ,    1.3 ,    5.6 ,    2.6 ,    9.5 ,   16.  ,    0.15,
          1.1 ,   21.  ,   22.5 ,    2.8 ,   13.  ,   13.5 ,   11.3 ,
          3.2 ,    3.7 ,    9.9 ,    6.5 ,    9.1 ,    3.6 ,    8.5 ,
         24.  ,   25.  ,   73.5 ,   67.5 ,   69.  ,   69.5 ,   53.  ,
         89.5 ,   48.5 ,   45.  ,   82.  ,   85.  ,   10.5 ,   14.  ,
          0.4 ,   12.3 ,   16.5 ,   17.  ,    3.9 ,    9.8 ,    0.78,
          9.3 ,   40.  ,    4.6 ,    7.8 ,    4.4 ,    0.25,   13.3 ,
          4.7 ,    4

In [307]:
# 1원짜리 거래 발견, 너무 거래값이 작은 값들은 삭제
pd.options.display.float_format = '{:.5f}'.format
df2['price'].describe()

count     489074.00000
mean       24203.24418
std       158542.65704
min            0.00000
25%         7750.00000
50%        16000.00000
75%        32000.00000
max     88005500.00000
Name: price, dtype: float64

In [308]:
df2 = df2.drop(df2[df2['price']<500].index)

In [309]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[가리비국산]가리비국산,5.00000,23000,경남 남해,normal,2021-03-01,3
1,[가리비국산]가리비국산,5.00000,25000,"경남 마산(고성,진해)",normal,2021-03-01,3
2,[가리비국산]가리비국산,5.00000,24000,"경남 마산(고성,진해)",normal,2021-03-01,3
3,[가오리수입]가오리수입,10.00000,51230,수입 아르헨티나,normal,2021-03-01,3
4,[가자미국산]가자미국산,15.00000,162000,국내산,normal,2021-03-01,3
...,...,...,...,...,...,...,...
489069,[토마토]토마토(기타),2.50000,5500,부산 강서구,normal,2021-03-30,7
489070,[감귤]감귤(금귤),0.50000,1100,제주 서귀포,normal,2021-03-31,7
489071,[감귤]감귤(금귤),10.00000,14000,제주 서귀포,normal,2021-03-31,7
489072,[당근]당근(당근),20.00000,9000,제주 제주시,normal,2021-03-31,7


In [310]:
# kg 당 가격 생성

df2['price_kg'] = round((df2['price']/df2['scale']),2)

### prd
- 품목명 정리
- 품목 종류 정리

In [311]:
# 대부분 [상품 종류]상품명 으로 정리되어 있는걸 알 수 있음
df2['prd'].value_counts()

[딸기]설향              16384
[오이]백다다기            13533
[딸기]설향(딸기)           8638
[시금치]시금치             7244
[딸기]딸기(설향)           6387
                    ...  
[생강]생강(기타)              1
[기타엽채]머우대               1
[기타엽채]기타엽채(얼갈이솎음        1
[포도]포도(기타)              1
[석류(수입)]석류(수입)          1
Name: prd, Length: 1237, dtype: int64

In [312]:
# 정리를 위해 상품명 삭제
def prd_sort(words):
  p = re.compile('[ㄱ-힣]+')
  word = re.search(p,words)
  return word.group()

df2['prd'] = df2['prd'].apply(prd_sort)

In [313]:
# 기타 품목 삭제
df2 = df2.drop(df2[df2['prd']=='기타'].index)

In [314]:
# 국산,수입산,깐,기타 삭제
def prd_sort2(words):
  p = re.compile('[^국산수입깐기타]+')
  word = re.search(p,str(words))
  return word.group()

df2["prd"] = df2["prd"].apply(prd_sort2)


In [315]:
# 몇몇 품목들 정리가 필요
a = sorted(df2['prd'].unique())
for i in range(len(a)):
  if i%15 != 0:
    print(a[i],end=' ')
  else:
    print(a[i],end=' ')
    print()

가리비 
가물치 가오리 가오리채 가자미 가지 각굴 갈치 감 감귤 감말랭이 감자 갑오징어 갓 개량조개 개조개 
갯가재 건고추 건채류 게 게지 겨자 고 고구마 고둥 고들빼 고등어 고사리 고추 고추잎 곤달비 
골뱅이 곶감 과실류 과일 과일과채류 과일류 과채 굴 귤 그레이프푸룻 근대 근채류 금감 깻잎 꼴뚜 
꽁치 꽈리고추 낙지 날치알 냉이 논우렁이 눈볼대 느 다랑어 다래 단감 달래 닭새우 당귀 당근 
대구 대추 대파 더덕 도라지 돈나물 돔 돗나물 동자개 동죽 두릅 두리안 딸 땅콩 라임 
람부탄 레몬 리치 마 마늘 만가닥 만감 맛 망고 망고스턴 망고스틴 매생이 머위대 메 메론 
메밀 명태 명태곤니 명태알 명태포 모시조개 모자반 무 무순 무우 문어바지락 미꾸라지 미나리 미더덕 미역 
민들레 민물새우 민어 바나나 바지락 박 밤 방울토마토 방풍 방풍나물 배 배추 백새우 백합 뱀장어 
뱅어 버섯 버섯류 버찌 벅굴 벗 병어 봄동 봄동배추 봉지굴 부세 부지갱이 부추 붉은고추 붕어 
붕장어 브로커리 브로코리 블루베리 비름 비트 빈스 빨간양배추 삐틀이 사과 삼 삼나물 삼치 상추 새꼬막 
새송이 새싹 새우 새우살 새조개 샐러리 생강 생고추 석류 선동오징어 세러리 세발나물 셀러리 소라 속새 
숙주나물 숫꽃게 시금치 신선초 실파 쌈채류 쑥 쑥갓 씀바귀 아귀 아보카도 아보카드 아스파라가스 아스파라거스 아욱 
알 알배 야자 양배추 양상추 양송이 양채 양채류 양파 어류 얼갈이 얼갈이배추 엄나무 연근 연어 
열무 염고등어 엽경채류 엽채 엽채류 오렌지 오만둥이 오이 오징어 오징어몸통 오징어원양 오징어채 완두콩 용과 우럭조개 
우렁 우렁쉥이 우렁이 우뭇가시리 우엉 원추리 유채 은어 임연 잉어 자몽 재첩 적겨자 적근대 적새우 
적채 전복 절단 절단낙지 조 조개 조개살 죽순 중하 쪽파 쭈꾸미 참나물 참다래 참외 참조 
채소 청경채 청어 체리 총각무 취나물 치커리 칠게 칼라후라워 칼리플라워 칼리후라워 케일 코라비 코코넛 콜라비 
콩 콩나물 키위 키조개 키조개꼭지 키조개날개 토마토 톳 파 파래김 파세리 파

In [316]:
### 예시
# df2[(df2['prd']=='유채') & (df2['price']==4500)]['prd'].unique()
# df2[df2['prd']=='유채']['prd'].unique()

In [317]:
### 원래 단어 뭐였는지 검색
# org_df[org_df['품목명'].str.contains('느')]['품목명'].unique()

In [318]:
### 현재 이름 검색
# df2[df2['prd']=='느']

In [319]:
###
# JG

# 고 -> 고수로 변경
df2.loc[df2["prd"]=="고","prd"] = "고수"
# 과실류, 과일류, 과채, 채소 -> 과채류로 통합
df2.loc[df2["prd"]=="과실류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일야채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일과채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일","prd"] = "과채류"
df2.loc[df2["prd"]=="과채","prd"] = "과채류"
df2.loc[df2["prd"]=="채소","prd"] = "과채류"
# 고들빼 -> 고들빼기로 변경
df2.loc[df2["prd"]=="고들빼","prd"] = "고들빼기"
# 꼴뚜 -> 꼴두기로 변경
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
# 그레이프푸룻 -> 자몽으로 통합
df2.loc[df2["prd"]=="그레이프푸룻","prd"] = "자몽"

# 느 -> 느타리버섯으로 통합
df2.loc[df2["prd"]=="느","prd"] = "느타리버섯"

# 대구 관련 품목들 정리
df2.loc[df2["prd"]=="대구고니","prd"] = "대구"
df2.loc[df2["prd"]=="대구머리","prd"] = "대구"
df2.loc[df2["prd"]=="대구포","prd"] = "대구"
# 듀리안 -> 두리안으로 통합
df2.loc[df2["prd"]=="듀리안","prd"] = "두리안"
# 딸 -> 딸기로 변경
df2.loc[df2["prd"]=="딸","prd"] = "딸기"

# 만가닥 -> 만가닥버섯으로 변경
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
# 맛 -> 맛조개로 변경
df2.loc[df2["prd"]=="맛","prd"] = "맛조개"
# 망고스턴 -> 망고스틴으로 수정
df2.loc[df2["prd"]=="망고스턴","prd"] = "망고스틴"
# 메 -> 메기로 변경
df2.loc[df2["prd"]=="메","prd"] = "메기"
# 명태 관련 품목들 정리
df2.loc[df2["prd"]=="명태곤니","prd"] = "명태"
df2.loc[df2["prd"]=="명태알","prd"] = "명태"
df2.loc[df2["prd"]=="명태포","prd"] = "명태"
# 무우 -> 무로 통합
df2.loc[df2["prd"]=="무우","prd"] = "무"

# 방풍 -> 방풍나물로 통합
df2.loc[df2["prd"]=="방풍","prd"] = "방풍나물"
# 버섯류 -> 버섯으로 변경
df2.loc[df2["prd"]=="버섯","prd"] = "버섯"
# 벗,버찌 -> 체리로 통합
df2.loc[df2["prd"]=="벗","prd"] = "체리"
df2.loc[df2["prd"]=="버찌","prd"] = "체리"
# 브로커리, 브로코리 -> 브로콜리로 통합
df2.loc[df2["prd"]=="브로커리","prd"] = "브로콜리"
df2.loc[df2["prd"]=="브로코리","prd"] = "브로콜리"

# 삼 -> 수삼으로 변경
df2.loc[df2["prd"]=="삼","prd"] = "수삼"
# 새우살 -> 새우로 변경
df2.loc[df2["prd"]=="새우살","prd"] = "새우"
# 세러리, 셀러리 -> 샐러리로 통합
df2.loc[df2["prd"]=="세러리","prd"] = "샐러리"
df2.loc[df2["prd"]=="셀러리","prd"] = "샐러리"
# 생고추 -> 고추로 통합
df2.loc[df2["prd"]=="생고추","prd"] = "고추"

# 아보카드 -> 아보카도로 변경
df2.loc[df2["prd"]=="아보카드","prd"] = "아보카도"
# 아스파라가스 -> 아스파라거스로 변경
df2.loc[df2["prd"]=="아스파라가스","prd"] = "아스파라거스"
# 알 -> 알타리무로 변경
df2.loc[df2["prd"]=="알","prd"] = "알타리무"
# 알배 -> 알배기로 변경
df2.loc[df2["prd"]=="알배","prd"] = "알배기"
# 양채 -> 양채류로 통합
df2.loc[df2["prd"]=="양채","prd"] = "양채류"
# 얼갈이 -> 얼갈이배추로 통합
df2.loc[df2["prd"]=="얼갈이","prd"] = "얼갈이배추"
# 엽채류, 엽채 -> 엽경채류로 통합
df2.loc[df2["prd"]=="엽채류","prd"] = "엽경채류"
df2.loc[df2["prd"]=="엽채","prd"] = "엽경채류"
# 오징어 관련 품목들 정리
df2.loc[df2["prd"]=="오징어다리","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어원양","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채","prd"] = "오징어"
df2.loc[df2["prd"]=="활복오징어","prd"] = "오징어"
# 옥, 풋옥 -> 옥수수로 정리
df2.loc[df2["prd"]=="옥","prd"] = "옥수수"
df2.loc[df2["prd"]=="풋옥","prd"] = "옥수수"
# 임연 -> 임연수어로 정리
df2.loc[df2["prd"]=="임연","prd"] = "임연수어"


# 절단 -> 절단쭈꾸미 -> 쭈꾸미로 정리
df2.loc[df2["prd"]=="절단","prd"] = "쭈꾸미"
# 조개살 -> 조개로 정리
df2.loc[df2["prd"]=="조개살","prd"] = "조개"

# 참조 -> 참조기로 정리
df2.loc[df2["prd"]=="참조","prd"] = "참조기"

# 칼라후라워,칼리플라워, 칼리후라워 -> 컬리플라워로 통합
df2.loc[df2["prd"]=="칼라후라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리플라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리후라워","prd"] = "컬리플라워"
# 코라비 -> 콜라비로 통합
df2.loc[df2["prd"]=="코라비","prd"] = "콜라비"
# 키조개날개 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개날개","prd"] = "키조개"

# 팽이 -> 팽이버섯으로 통합
df2.loc[df2["prd"]=="팽이","prd"] = "팽이버섯"
# 파세리 -> 파슬리로 변경
df2.loc[df2["prd"]=="파세리","prd"] = "파슬리"
# 표고 -> 표고버섯으로 통합
df2.loc[df2["prd"]=="표고","prd"] = "표고버섯"
# 포장바지락 -> 바지락으로 변경
df2.loc[df2["prd"]=="포장바지락","prd"] = "바지락"

### 일단 여기까지 실행
# JY
#각굴 -> 굴로 통합
df2.loc[df2["prd"]=="각굴","prd"] = "굴"
#가오리채 -> 가오리로 통합
df2.loc[df2["prd"]=="가오리채","prd"] = "가오리"
# 게지 -> 키조개로 통합
df2.loc[df2["prd"]=="게지","prd"] = "키조개"
#고 -> 고구마로 통합
df2.loc[df2["prd"]=="고","prd"] = "고구마"
#꼴뚜 -> 꼴뚜기로 통합
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
#돗나물 -> 돌나물로 통합
df2.loc[df2["prd"]=="돗나물","prd"] = "돌나물"
#레드쉬 -> 레디쉬로 통합
df2.loc[df2["prd"]=="레드쉬","prd"] = "레디쉬"
#만가닥 -> 만가닥버섯으로 통합
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
#메 -> 메기로 통합
df2.loc[df2["prd"]=="메","prd"] = "메기"
#봄동 -> 봄동배추로 통합
df2.loc[df2["prd"]=="봄동","prd"] = "봄동배추"
#봉지굴 -> 굴로 통합
df2.loc[df2["prd"]=="봉지굴","prd"] = "굴"
#박 -> 수박으로 통합
df2.loc[df2["prd"]=="박","prd"] = "수박"
#백조 -> 조기로 통합 
df2.loc[df2["prd"]=="백조","prd"] = "조기"
#벅굴 -> 벚굴로 통합
df2.loc[df2["prd"]=="벅굴","prd"] = "벚굴"
#비 -> 비타민으로 통합
df2.loc[df2["prd"]=="비","prd"] = "비타민"
#빈스 -> 콩으로 통합
df2.loc[df2["prd"]=="빈스","prd"] = "콩"
#삐틀이 -> 고둥으로 통합
df2.loc[df2["prd"]=="삐틀이","prd"] = "고둥"
#새송이 -> 새송이버섯으로 통합
df2.loc[df2["prd"]=="새송이","prd"] = "새송이버섯"
#세트 -> 과채류로 통합
df2.loc[df2["prd"]=="세트","prd"] = "과채류"
#알 -> 알로애로 통합
df2.loc[df2["prd"]=="알","prd"] = "알로에"
#엄나무 -> 음나무로 통합
df2.loc[df2["prd"]=="엄나무","prd"] = "음나무"
#염고등어 -> 고등어로 통합
df2.loc[df2["prd"]=="염고등어","prd"] = "고등어"
#오만둥이 -> 미더덕으로 통합
df2.loc[df2["prd"]=="오만둥이","prd"] = "미더덕"
#오징어 관련 품목들 정리
df2.loc[df2["prd"]=="선동오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어몸통","prd"] = "오징어"
#우렁쉥이 -> 멍게로 통합
df2.loc[df2["prd"]=="우렁쉥이","prd"] = "멍게"
#우뭇가시리 -> 우뭇가사리로 통합
df2.loc[df2["prd"]=="우뭇가시리","prd"] = "우뭇가사리"
#적채 -> 적양배추로 통합
df2.loc[df2["prd"]=="적채","prd"] = "적양배추"
#절단낙지 -> 낙지로 통합
df2.loc[df2["prd"]=="절단낙지","prd"] = "낙지"
#조 -> 조개로 통합
df2.loc[df2["prd"]=="조","prd"] = "조개"
#청어원양 -> 청어로 통합
df2.loc[df2["prd"]=="청어원양","prd"] = "청어"
#키조개꼭지 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개꼭지","prd"] = "키조개"

# 암꽃게, 숫꽃게 -> 꽃게로 통합
df2.loc[df2["prd"]=="암꽃게","prd"] = "꽃게"
df2.loc[df2["prd"]=="숫꽃게","prd"] = "꽃게"
# 우렁 -> 우렁이로 통합
df2.loc[df2["prd"]=="우렁","prd"] = "우렁이"
# 알로애 -> 알로에로 변경
df2.loc[df2["prd"]=="알로애","prd"] = "알로에"
# 피마자잎 -> 피마자로 변경
df2.loc[df2["prd"]=="피미자잎","prd"] = "피마자"
# 조미제품 삭제
df2.drop(df2[df2['prd']=='조미제품'].index,inplace=True)

In [320]:
# 기존 products 리스트와 비교
ex_prd = pd.read_csv("../products.csv")
df2[~df2['prd'].isin(ex_prd['prd'])]['prd'].unique()

array([], dtype=object)

In [321]:
# 물품 리스트 최신화
new_prd = pd.DataFrame(df2['prd'].unique(),columns=['prd'])
# products = ex_prd.append(new_prd[~new_prd['prd'].isin(ex_prd['prd'])],ignore_index=True)
products = ex_prd.append(new_prd,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_8411/2619460125.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products = ex_prd.append(new_prd,ignore_index=True)


In [322]:
products.drop(columns=['Unnamed: 0'],inplace=True)
products.drop_duplicates(inplace=True)
products.reset_index(inplace=True,drop=True)

In [323]:
# 다음 작업을 위해 저장
products.to_csv('../products.csv')

In [324]:
# 전체 거래량과 품목량
# 비교를 위해 분포 파악
# 거래량이 너무 적은 품목은 삭제할 필요성이 있음

prd_lst = (df2['prd'].value_counts()).to_frame()
prd_lst.describe()

,prd
count,237.00000
mean,2059.81013
std,5799.91850
min,1.00000
25%,10.00000
50%,98.00000
75%,1182.00000
max,54439.00000


In [325]:
# 50개 미만 품목은 비교가 힘드므로 삭제
# 줄어든 종류만 남기기

prd_lst = prd_lst[prd_lst['prd']>=50]
prd_lst.reset_index(inplace=True)
prd_lst = prd_lst.rename(columns = {'index':'prd','prd':'count'})

In [326]:
df3 = df2.merge(prd_lst)
df3[df3['count']>=50]['prd'].value_counts()

딸기     54439
버섯     39031
오이     26789
토마토    24571
양채류    19021
       ...  
콩나물       58
알배기       54
개조개       52
겨자        52
자몽        50
Name: prd, Length: 139, dtype: int64

In [327]:
# 줄어든 것 확인했으므로 count column은 삭제
df3 = df3.drop(columns='count')

### scale

In [328]:
np.set_printoptions(precision=2, suppress=True)
df3['scale'].unique()

array([   5.  ,   10.  ,   15.  ,    4.  ,    4.5 ,   18.  ,   16.  ,
          8.  ,    5.01,   13.  ,   65.  ,   57.  ,    0.4 ,   28.  ,
          7.  ,   12.  ,   24.  ,   49.  ,    3.  ,    2.  ,    9.  ,
         20.  ,    6.  ,    7.5 ,    1.  ,    0.5 ,    1.5 ,    0.8 ,
          2.5 ,    9.5 ,   11.2 ,    4.2 ,    9.6 ,    4.8 ,    3.75,
         10.2 ,    5.04,   23.  ,   22.3 ,   17.  ,    2.4 ,    1.7 ,
          1.2 ,    5.4 ,   40.  ,   46.  ,    3.5 ,   11.  ,    3.6 ,
          0.2 ,    6.75,    0.3 ,    0.1 ,   14.  ,   12.5 ,    8.6 ,
          3.9 ,    5.1 ,    8.5 ,    4.3 ,    9.7 ,    6.7 ,    1.6 ,
          6.5 ,    7.2 ,    4.6 ,    4.9 ,    5.3 ,    5.6 ,    7.1 ,
          0.05,    0.25,    0.13,   10.5 ,   51.  ,    0.15,    0.7 ,
         22.  ,   19.  ,   26.  ,    8.3 ,   21.  ,    5.5 ,    0.9 ,
          2.3 ,    1.1 ,    3.4 ,    3.7 ,    4.4 ,    4.7 ,    1.8 ,
          2.2 ,    1.9 ,    2.1 ,    0.78,    2.8 ,    3.8 ,    3.2 ,
          2.7 ,    1

In [329]:
# 50kg 이상 거래된 건들 이상치라 판단하고 확인 진행
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
1850,가지,65.00000,377000,경남 진주시,certified,2021-03-05,1,5800.00000
2043,가지,57.00000,330600,경남 진주시,certified,2021-03-08,1,5800.00000
65793,양채류,51.00000,10500,제주 제주시,normal,2021-03-13,1,205.88000
65802,양채류,51.00000,12500,제주 제주시,normal,2021-03-13,1,245.10000
117997,딸기,100.00000,11000,충남 논산시,normal,2021-03-12,1,110.00000
117999,딸기,100.00000,7000,충남 논산시,normal,2021-03-12,1,70.00000
176369,배추,6978.00000,4186800,전남 완도군,normal,2021-03-05,1,600.00000
176370,배추,2880.00000,1728000,전남 완도군,normal,2021-03-05,1,600.00000
176371,배추,5920.00000,3552000,전남 해남군,normal,2021-03-05,1,600.00000
194555,버섯,198.00000,643500,충남 보령시,normal,2021-03-08,1,3250.00000


In [330]:
df3[(df3['prd']=='가지')&(df3['scale']>30)]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
1850,가지,65.00000,377000,경남 진주시,certified,2021-03-05,1,5800.00000
2043,가지,57.00000,330600,경남 진주시,certified,2021-03-08,1,5800.00000
4106,가지,49.00000,284200,경남 진주시,certified,2021-03-31,1,5800.00000


In [331]:
# 스케일 오표기된 항목 수정
# 가지는 가격이 평균보다 비싸긴 하나 거래 불가능한 정도는 아니라 판단


# 나머지는 이상치로 판단하고 모두 삭제
df3 = df3.drop(df3[(df3['scale']>=50)&(df3['prd']!='가지')].index)

In [332]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg
1850,가지,65.00000,377000,경남 진주시,certified,2021-03-05,1,5800.00000
2043,가지,57.00000,330600,경남 진주시,certified,2021-03-08,1,5800.00000


### origin

In [333]:
# 도,시, 국산, 수입 표기 분류 필요
df3['origin'].unique()

array(['경남 남해', '경남 마산(고성,진해)', '전남 여수', '경남 통영', '국내산', '수입 러시아',
       '전북 군산', '수입 미국', '제주 제주도', '대구 달성군', '전북 군산시', '경기 여주시', '경남 김해시',
       '경남 밀양시', '광주시 광산구', '경남 진주시', '경남 창원시 의창구', '경남 고성군', '경남 창녕군',
       '경남 함안군', '경북 고령군', '경북 상주시', '경북 성주군', '광주 광산구', '전남 나주시',
       '경기 여주군', '전북 임실군', '경남 거제시', '전북 익산시', '경기 안성시 일죽면', '경기 하남시',
       '경기 이천시', '경기 용인시', '경기도 하남시', '경기 양평군', '전북 전주시', '충남 부여군',
       '경남 창원시', '전남 광양시', '서울', '경남', '충남 금산군', '경기 광주시', '경남 의령군',
       '경남 밀양시 단장면', '전북 완주군', '경남 진주시 금곡면', '경기 안성시 고삼면', '전북 군산시 개정면',
       '경기 안성시', '충남 예산군', '서울 강동구', '전남 고흥(나로도)', '경남 삼천포', '충남 서산',
       '수입 세네갈', '수입 기타', '수입 중국', '제주', '제주 서귀포시', '제주 제주시', '제주 서귀포',
       '제주 남제주군', '제주특별자치도 서귀', '제주도 서귀포시', '서울시 강서구', '제주 북제주군',
       '전남 담양군', '전남 고흥군', '제주도 제주시', '경남 산청군', '전남 해남군', '제주 서귀포시(2)',
       '제주 남제주', '수입산 미국', '충남 논산시', '경기 평택시 고덕면', '전남 무안군', '제주특별자치도 제주',
       '미국', '서울특별시 강남구', '부산시 기장군', '강원 평창군', '강원 홍천군', '경북 예천군',
       '충남 당진시',

In [334]:
# 해당 조건에 맞는 값 추출

splits = df3['origin'].str.split(' ')
cacul = splits.apply(lambda x: pd.Series(x))
df66 = pd.merge(df3,cacul, left_index=True, right_index=True, how='inner')
df66

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,0,1,2,3
0,가리비,5.00000,23000,경남 남해,normal,2021-03-01,3,4600.00000,경남,남해,NaN,NaN
1,가리비,5.00000,25000,"경남 마산(고성,진해)",normal,2021-03-01,3,5000.00000,경남,"마산(고성,진해)",NaN,NaN
2,가리비,5.00000,24000,"경남 마산(고성,진해)",normal,2021-03-01,3,4800.00000,경남,"마산(고성,진해)",NaN,NaN
3,가리비,5.00000,25000,전남 여수,normal,2021-03-02,3,5000.00000,전남,여수,NaN,NaN
4,가리비,5.00000,23000,경남 남해,normal,2021-03-02,3,4600.00000,경남,남해,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
486894,쑥,5.00000,8000,전남 신안군,normal,2021-03-31,1,1600.00000,전남,신안군,NaN,NaN
486895,쑥,6.00000,40500,충남 보령시,normal,2021-03-31,1,6750.00000,충남,보령시,NaN,NaN
486896,쑥,5.00000,32500,충남 태안군,normal,2021-03-31,1,6500.00000,충남,태안군,NaN,NaN
486897,쑥,4.00000,32000,충남 당진시,normal,2021-03-31,1,8000.00000,충남,당진시,NaN,NaN


In [335]:
# 세부 카테고리 정리 (state, city 편집 시 여기서 시작)

df4 = df66.rename(columns = {0:'state',1:'city'})

# 2,3은 필요없는 정보이므로 삭제
df4 = df4.drop(columns=[2,3])

In [336]:
df4['state'].unique()

array(['경남', '전남', '국내산', '수입', '전북', '제주', '대구', '경기', '광주시', '경북', '광주',
       '경기도', '충남', '서울', '제주특별자치도', '제주도', '서울시', '수입산', '미국', '서울특별시',
       '부산시', '강원', '강원도', '전라북도', '부산', '충북', '인천시', '인천', '가락동', '성남시',
       '중국', '대만', '포항시', '대전', '베트남', '충청남도', '필리핀', '전라남도', '경상남도',
       '청주시', '세종시', '경북/문경시', '태국', '세종', '원양산', '인천광역시', '뉴질랜드', '울산',
       '대구시', '칠레', '페루', '호주', '오스트레일', '전주시', '불가리아', '멕시코', '이탈리아',
       '충북청주시상당구', '전북전주시완산구', '타이', '부천시', '과테말라', '콜롬비아', '에쿠아도르',
       '인도네시아', '에콰돌', '에콰도르', '기타외국', '코스타리카', '스페인', '이스라엘'],
      dtype=object)

In [337]:
# 시,도 제거
def name_sort(words):
  p = re.compile('[^시도]+')
  word = re.search(p,str(words))
  return word.group()

df4["state"] = df4["state"].apply(name_sort)

# 오타도 제거

def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["state"] = df4["state"].apply(name_sort2)

In [338]:
df4["state"].unique()

array(['경남', '전남', '국내산', '수입', '전북', '제주', '대구', '경기', '광주', '경북', '충남',
       '서울', '제주특별자치', '수입산', '미국', '서울특별', '부산', '강원', '전라북', '충북', '인천',
       '가락동', '성남', '중국', '대만', '포항', '대전', '베트남', '충청남', '필리핀', '전라남',
       '경상남', '청주', '세종', '경북/문경', '태국', '원양산', '인천광역', '뉴질랜드', '울산',
       '칠레', '페루', '호주', '오스트레일', '전주', '불가리아', '멕', '이탈리아', '충북청주',
       '전북전주', '타이', '부천', '과테말라', '콜롬비아', '에쿠아', '인', '에콰돌', '에콰',
       '기타외국', '코스타리카', '스페인', '이스라엘'], dtype=object)

In [339]:
# state
# 특이케이스 제거

# 국내
df4.loc[df4["state"]=="국내산","city"] = "미상"
df4.loc[df4["state"]=="국내산","state"] = "국산"
df4.loc[df4["state"]=="원양산","city"] = "미상"

## 도단위
df4.loc[df4["state"]=="충청북","state"] = "충북"
df4.loc[df4["state"]=="충청남","state"] = "충남"
df4.loc[df4["state"]=="전라북","state"] = "전북"
df4.loc[df4["state"]=="전라남","state"] = "전남"
df4.loc[df4["state"]=="경상북","state"] = "경북"
df4.loc[df4["state"]=="경상남","state"] = "경남"

## 시단위
### 광역시

df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="대구","city"] = "대구"
df4.loc[df4["state"]=="서울특별","city"] = "서울"
df4.loc[df4["state"]=="서울특별","state"] = "서울"
df4.loc[df4["state"]=="인천광역","city"] = "인천"
df4.loc[df4["state"]=="인천광역","state"] = "인천"
df4.loc[df4["state"]=="부산","city"] = "부산"
df4.loc[df4["state"]=="가락동","city"] = "서울"
df4.loc[df4["state"]=="가락동","state"] = "서울"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="제주","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","state"] = "제주"
df4.loc[df4["state"]=="제주특별자","city"] = "제주"
df4.loc[df4["state"]=="제주특별자","state"] = "제주"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="울산","city"] = "울산"
df4.loc[df4["state"]=="울산","state"] = "울산"

### 일반시
df4.loc[df4["state"]=="성남","city"] = "성남"
df4.loc[df4["state"]=="성남","state"] = "경기"
df4.loc[df4["state"]=="전주","city"] = "전주"
df4.loc[df4["state"]=="전주","state"] = "전북"
df4.loc[df4["state"]=="청주","city"] = "청주"
df4.loc[df4["state"]=="청주","state"] = "충북"
df4.loc[df4["state"]=="수원","city"] = "수원"
df4.loc[df4["state"]=="수원","state"] = "경기"
df4.loc[df4["state"]=="부천","city"] = "부천"
df4.loc[df4["state"]=="부천","state"] = "경기"
df4.loc[df4["state"]=="경기고양","city"] = "고양"
df4.loc[df4["state"]=="경기고양","state"] = "경기"
df4.loc[df4["state"]=="충북청주","city"] = "청주"
df4.loc[df4["state"]=="충북청주","state"] = "충북"
df4.loc[df4["state"]=="전북전주","city"] = "전주"
df4.loc[df4["state"]=="전북전주","state"] = "전북"
df4.loc[df4["state"]=="경북/문경","city"] = "문경"
df4.loc[df4["state"]=="경북/문경","state"] = "경북"
df4.loc[df4["state"]=="포항","city"] = "포항"
df4.loc[df4["state"]=="포항","state"] = "경북"


# 외국
## 아시아
df4.loc[df4["state"]=="칠레","city"] = "칠레"
df4.loc[df4["state"]=="칠레","state"] = "수입"
df4.loc[df4["state"]=="일본","city"] = "일본"
df4.loc[df4["state"]=="일본","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="우즈베크","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베크","state"] = "수입"
df4.loc[df4["state"]=="이스라엘","city"] = "이스라엘"
df4.loc[df4["state"]=="이스라엘","state"] = "수입"
df4.loc[df4["state"]=="터키","city"] = "터키"
df4.loc[df4["state"]=="터키","state"] = "수입"
df4.loc[df4["state"]=="중국","city"] = "중국"
df4.loc[df4["state"]=="중국","state"] = "수입"
df4.loc[df4["state"]=="타이","city"] = "태국"
df4.loc[df4["state"]=="타이","state"] = "수입"
df4.loc[df4["state"]=="태국","city"] = "태국"
df4.loc[df4["state"]=="태국","state"] = "수입"
df4.loc[df4["state"]=="필리핀","city"] = "필리핀"
df4.loc[df4["state"]=="필리핀","state"] = "수입"
df4.loc[df4["state"]=="우즈베키스탄","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베키스탄","state"] = "수입"
df4.loc[df4["state"]=="대만","city"] = "대만"
df4.loc[df4["state"]=="대만","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="이란","city"] = "이란"
df4.loc[df4["state"]=="이란","state"] = "수입"
df4.loc[df4["state"]=="러","city"] = "러시아"
df4.loc[df4["state"]=="러","state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"city"] = "인도네시아"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"city"] = "인도"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"state"] = "수입"

## 아메리카

df4.loc[df4["state"]=="캘리포니아","city"] = "미국"
df4.loc[df4["state"]=="캘리포니아","state"] = "수입"
df4.loc[df4["state"]=="포클랜드","city"] = "영국"
df4.loc[df4["state"]=="포클랜드","state"] = "수입"
df4.loc[df4["state"]=="콜롬비아","city"] = "콜롬비아"
df4.loc[df4["state"]=="콜롬비아","state"] = "수입"
df4.loc[df4["state"]=="에콰","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰","state"] = "수입"
df4.loc[df4["state"]=="에쿠아","city"] = "에콰도르"
df4.loc[df4["state"]=="에쿠아","state"] = "수입"
df4.loc[df4["state"]=="에콰돌","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰돌","state"] = "수입"
df4.loc[df4["state"]=="과테말라","city"] = "과테말라"
df4.loc[df4["state"]=="과테말라","state"] = "수입"
df4.loc[df4["state"]=="코스타리카","city"] = "코스타리카"
df4.loc[df4["state"]=="코스타리카","state"] = "수입"
df4.loc[df4["state"]=="멕","city"] = "멕시코"
df4.loc[df4["state"]=="멕","state"] = "수입"
df4.loc[df4["state"]=="페루","city"] = "페루"
df4.loc[df4["state"]=="페루","state"] = "수입"
df4.loc[df4["state"]=="미국","city"] = "미국"
df4.loc[df4["state"]=="미국","state"] = "수입"
df4.loc[df4["state"]=="아르헨티나","city"] = "아르헨티나"
df4.loc[df4["state"]=="아르헨티나","state"] = "수입"

## 유럽
df4.loc[df4["state"]=="스페인","city"] = "스페인"
df4.loc[df4["state"]=="스페인","state"] = "수입"
df4.loc[df4["state"]=="우크라이나","city"] = "우크라이나"
df4.loc[df4["state"]=="우크라이나","state"] = "수입"
df4.loc[df4["state"]=="이탈리아","city"] = "이탈리아"
df4.loc[df4["state"]=="이탈리아","state"] = "수입"
df4.loc[df4["state"]=="불가리아","city"] = "불가리아"
df4.loc[df4["state"]=="불가리아","state"] = "수입"

## 아프리카
df4.loc[df4["state"]=="남아프리카","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카","state"] = "수입"
df4.loc[df4["state"]=="남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카공화국","state"] = "수입"
df4.loc[df4["state"]=="수입남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="수입남아프리카공화국","state"] = "수입"


## 오세아니아
df4.loc[df4["state"]=="뉴질랜드","city"] = "뉴질랜드"
df4.loc[df4["state"]=="뉴질랜드","state"] = "수입"
df4.loc[df4["state"]=="오스트레일","city"] = "호주"
df4.loc[df4["state"]=="오스트레일","state"] = "수입"
df4.loc[df4["state"]=="오스트레일리아","city"] = "호주"
df4.loc[df4["state"]=="오스트레일리아","state"] = "수입"
df4.loc[df4["state"]=="호주","city"] = "호주"
df4.loc[df4["state"]=="호주","state"] = "수입"
df4.loc[df4["state"]=="통가","city"] = "통가"
df4.loc[df4["state"]=="통가","state"] = "수입"
df4.loc[df4["state"]=="뉴칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴칼레","state"] = "수입"
df4.loc[df4["state"]=="뉴","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴","state"] = "수입"
df4.loc[df4["state"]=="누벨칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="누벨칼레","state"] = "수입"
# 기타
df4.loc[df4["state"]=="수입산","state"] = "수입"
df4.loc[df4["state"]=="기타외국","city"] = "미상"
df4.loc[df4["state"]=="기타외국","state"] = "수입"

In [340]:
df4['state'].unique()

array(['경남', '전남', '국산', '수입', '전북', '제주', '대구', '경기', '광주', '경북', '충남',
       '서울', '부산', '강원', '충북', '인천', '대전', '세종', '원양산', '울산'],
      dtype=object)

In [341]:
# city
# 우선 결측치 제거
df4.loc[df4["city"].isnull(),"city"] = "미상"

In [342]:
# 이후 시,군 등 제거
def name_sort(words):
  p = re.compile('[^시군]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort)

In [343]:
# \t 도 제거
def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort2)

In [344]:
df4['city'].unique()

array(['남해', '마산(고성,진해)', '여수', '통영', '미상', '러', '산', '미국', '제주', '대구',
       '여주', '김해', '밀양', '광산구', '진주', '창원', '고성', '창녕', '함안', '고령', '상주',
       '성주', '나주', '임실', '거제', '익산', '안성', '하남', '이천', '용인', '양평', '전주',
       '부여', '광양', '금산', '광주', '의령', '완주', '예산', '강동구', '고흥(나로도)', '삼천포',
       '서산', '세네갈', '기타', '중국', '강서구', '담양', '고흥', '산청', '해남', '논산', '평택',
       '무안', '서울', '부산', '평창', '홍천', '예천', '당진', '정선', '남원', '부안', '보성',
       '태안', '강릉', '화천', '홍성', '구례', '순천', '천안', '포천', '거창', '구미', '김제',
       '남구', '곡성', '김천', '횡성', '영천', '인제', '영주', '춘천', '서천', '합천', '원주',
       '공주', '남양주', '안동', '중구', '봉화', '청양', '영암', '충주', '칠곡', '문경', '송파구',
       '베트남', '장항(서천)', '바레인', '강화', '고창', '아산', '음성', '고양', '마산', '서구',
       '하동', '진안', '영동', '강진', '장성', '미추홀구', '성남', '제천', '흥', '양주', '광명',
       '안산', '중랑구', '도봉구', '구리', '동대문구', '진천', '양천구', '의왕', '경산', '괴산',
       '의정부', '화성', '파주', '신안', '영광', '철원', '영양', '청도', '대만', '포항', '영덕',
       '진도', '함평', '목포', '함양', '화순', '순창', '보

In [345]:
# 나머지 케이스 정리

# 국내
df4.loc[df4["city"]=="기타","city"] = "미상"
## 일반시
df4.loc[df4["city"]=="위","city"] = "군위"
df4.loc[df4["city"]=="고흥(나로도)","city"] = "고흥"
df4.loc[df4["city"]=="산","city"] = "군산"
df4.loc[df4["city"]=="장항(서천)","city"] = "서천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="순천(광양)","city"] = "순천"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="화성(평택)","city"] = "화성"
df4.loc[df4["city"]=="흥","city"] = "시흥"
df4.loc[df4["city"]=="목포(무안)","city"] = "목포"
df4.loc[df4["city"]=="목포(무안)","state"] = "전남"
df4.loc[df4["city"]=="미금","city"] = "성남"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="남"),"city"] = "남원"
df4.loc[(df4['prd']=="호박") & (df4['state']=='전북'),"city"] = "전주"
## 광역시
df4.loc[df4["city"]=="강서구","city"] = "서울"
df4.loc[df4["city"]=="은평구","city"] = "서울"
df4.loc[df4["city"]=="강동구","city"] = "서울"
df4.loc[df4["city"]=="마포구","city"] = "서울"
df4.loc[df4["city"]=="중구","city"] = "서울"
df4.loc[df4["city"]=="송파구","city"] = "서울"
df4.loc[df4["city"]=="도봉구","city"] = "서울"
df4.loc[df4["city"]=="중랑구","city"] = "서울"
df4.loc[df4["city"]=="양천구","city"] = "서울"
df4.loc[df4["city"]=="성동구","city"] = "서울"
df4.loc[df4["city"]=="강남구","city"] = "서울"
df4.loc[df4["city"]=="동대문구","city"] = "서울"
df4.loc[df4["city"]=="영등포구","city"] = "서울"
df4.loc[df4["city"]=="영등포","city"] = "서울"
df4.loc[df4["city"]=="서초구","city"] = "서울"
df4.loc[df4["city"]=="구로구","city"] = "서울"
df4.loc[df4["city"]=="동대문","city"] = "서울"
df4.loc[df4["city"]=="금천구","city"] = "서울"

df4.loc[df4["city"]=="광산구","city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="남구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="북구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="서구"),"city"] = "광주"
df4.loc[df4["city"]=="동구","city"] = "광주"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="남구"),"city"] = "인천"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="서구"),"city"] = "인천"
df4.loc[df4["city"]=="미추홀","city"] = "인천"
df4.loc[df4["city"]=="부평구","city"] = "인천"
df4.loc[df4["city"]=="남동구","city"] = "인천"
df4.loc[df4["city"]=="미추홀구","city"] = "인천"
df4.loc[df4["city"]=="계양구","city"] = "인천"
df4.loc[df4["city"]=="마산(고성,진해)","city"] = "창원"
df4.loc[(df4["state"]=="부산")&(df4["city"]=="북구"),"city"] = "부산"
df4.loc[df4["city"]=="오포","city"] = "부산"
df4.loc[df4["city"]=="부산진","city"] = "부산"
df4.loc[df4["city"]=="사상구","city"] = "부산"
df4.loc[df4["city"]=="부산진구","city"] = "부산"
df4.loc[df4["city"]=="북제주","city"] = "제주"
df4.loc[df4["city"]=="남제주","city"] = "제주"

# 해외
## 아시아
df4.loc[df4["city"]=="러","city"] = "러시아"
df4.loc[df4["city"]=="말레이지아","city"] = "말레이시아"
df4.loc[df4["city"]=="타이","city"] = "태국"
df4.loc[df4["city"]=="인도네","city"] = "인도네시아"
## 아메리카
df4.loc[df4["city"]=="멕","city"] = "멕시코"
## 아프리카
df4.loc[df4["city"]=="남아공산","city"] = "남아공"
## 유럽
## 오세아니아

In [346]:
# 기존 city 리스트와 비교
ex_city = pd.read_csv("../cities.csv")
df4[~df4['city'].isin(ex_city['city'])]['city'].unique()

array(['불가리아'], dtype=object)

In [347]:
# 리스트 최신화
new_city = pd.DataFrame(df4['city'].unique(),columns=['city'])
cities = ex_city.append(new_city,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_8411/1856950528.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = ex_city.append(new_city,ignore_index=True)


In [348]:
cities.drop(columns=['Unnamed: 0'],inplace=True)
cities.drop_duplicates(inplace=True)
cities.reset_index(inplace=True,drop=True)

In [349]:
cities

,city
0,남해
1,창원
2,중국
3,통영
4,여수
...,...
192,연천
193,칠곡
194,옹진
195,캄보디아


In [350]:
cities.to_csv('../cities.csv')

In [351]:
# 처리 완료하였으니 df4는 백업으로 두고 df5 활용
df5 = df4.drop(columns='origin')

# 전처리 결과

# 파일 저장

In [352]:
df5.to_csv('../data_proceed/202103_proceed.csv')